In [2]:
import struct
import os
# Open the file in binary mode

safetensor_file = "./net.safetensors"

def get_meta_size(file):
    with open(file, 'rb') as f:
        # Read the first 8 bytes
        bytes_data = f.read(8)
        # print them as uint8 integers, convert hex to decimal
        print(" ".join([str(x) for x in bytes_data]))
        # Unpack the bytes to a 64-bit unsigned integer
        meta_size = struct.unpack('Q', bytes_data)[0]
        return meta_size

print(get_meta_size(safetensor_file))

240 16 3 0 0 0 0 0
200944


In [12]:
from safetensors.numpy import load_file

loaded = load_file(safetensor_file)

In [15]:
# for ech k, v, print the size of v in megabytes
max_weight_size = 0
for k, v in loaded.items():
    print(k, v.shape, v.dtype, v.nbytes / 1024 / 1024, "MB")
    max_weight_size = max(max_weight_size, v.nbytes / 1024 / 1024)


alphas_cumprod (1000,) float32 0.003814697265625 MB
cond_stage_model.transformer.text_model.embeddings.position_embedding.vocab_counter (1, 1, 77) float32 0.000293731689453125 MB
cond_stage_model.transformer.text_model.embeddings.position_embedding.weight (77, 768) float32 0.2255859375 MB
cond_stage_model.transformer.text_model.embeddings.token_embedding.vocab_counter (1, 1, 49408) float32 0.1884765625 MB
cond_stage_model.transformer.text_model.embeddings.token_embedding.weight (49408, 768) float32 144.75 MB
cond_stage_model.transformer.text_model.encoder.layers.0.layer_norm1.bias (768,) float32 0.0029296875 MB
cond_stage_model.transformer.text_model.encoder.layers.0.layer_norm1.weight (768,) float32 0.0029296875 MB
cond_stage_model.transformer.text_model.encoder.layers.0.layer_norm2.bias (768,) float32 0.0029296875 MB
cond_stage_model.transformer.text_model.encoder.layers.0.layer_norm2.weight (768,) float32 0.0029296875 MB
cond_stage_model.transformer.text_model.encoder.layers.0.mlp.f

In [13]:
loaded['alphas_cumprod']

array([0.99915   , 0.998296  , 0.9974381 , 0.9965762 , 0.99571025,
       0.9948404 , 0.9939665 , 0.9930887 , 0.9922069 , 0.9913211 ,
       0.9904313 , 0.98953754, 0.9886398 , 0.9877381 , 0.9868324 ,
       0.98592263, 0.98500896, 0.9840913 , 0.9831696 , 0.982244  ,
       0.98131436, 0.9803808 , 0.97944313, 0.97850156, 0.977556  ,
       0.9766064 , 0.97565293, 0.9746954 , 0.9737339 , 0.9727684 ,
       0.97179896, 0.97082555, 0.96984816, 0.96886677, 0.9678814 ,
       0.96689206, 0.96589875, 0.9649015 , 0.96390027, 0.9628951 ,
       0.9618859 , 0.96087277, 0.95985574, 0.95883465, 0.9578097 ,
       0.95678073, 0.95574784, 0.954711  , 0.95367026, 0.9526256 ,
       0.9515769 , 0.95052433, 0.94946784, 0.94840735, 0.947343  ,
       0.94627476, 0.9452025 , 0.9441264 , 0.9430464 , 0.9419625 ,
       0.9408747 , 0.939783  , 0.9386874 , 0.93758786, 0.9364845 ,
       0.93537724, 0.9342661 , 0.9331511 , 0.9320323 , 0.9309096 ,
       0.929783  , 0.9286526 , 0.9275183 , 0.9263802 , 0.92523

In [8]:
import os
import struct

import argparse
from collections import namedtuple
from pathlib import Path
from typing import Any, List, NamedTuple

import numpy as np

from tinygrad.ops import Device
from tinygrad.nn.state import load_state_dict, torch_load, get_state_dict, safe_save, safe_load_metadata
from tinygrad.tensor import Tensor
from extra.export_model import jit_model, compile_net

from examples.stable_diffusion import (
  StableDiffusion,
  download_file,
)
from examples.controlnet import (
  ControlNetUNetModel, 
  ControlNetModel, 
  ControlNetStableDiffusion
)
FILENAME_DIFFUSION = "/Users/jasonx/Dropbox/repos/web-control-net/tinygrad/weights/sd-v1-4.ckpt"
FILENAME_CONTROLNET = "/Users/jasonx/Dropbox/repos/web-control-net/tinygrad/weights/sd-controlnet-canny.bin"
print(FILENAME_DIFFUSION)
download_file(
'https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt', FILENAME_DIFFUSION)
state_dict_diffusion = torch_load(FILENAME_DIFFUSION)["state_dict"]
diffusion_model = StableDiffusion()
diffusion_model.model = namedtuple("DiffusionModel", ["diffusion_model"])(
diffusion_model=ControlNetUNetModel())
load_state_dict(diffusion_model, state_dict_diffusion, strict=False)
download_file(
'https://huggingface.co/lllyasviel/sd-controlnet-canny/resolve/main/diffusion_pytorch_model.bin', FILENAME_CONTROLNET)
state_dict_controlnet = torch_load(FILENAME_CONTROLNET)
controlnet = ControlNetModel(cross_attention_dim=768)
load_state_dict(controlnet, state_dict_controlnet, strict=False)

model = ControlNetStableDiffusion(diffusion_model, controlnet)

/Users/jasonx/Dropbox/repos/web-control-net/tinygrad/weights/sd-v1-4.ckpt


python(74186) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
ram used:  4.26 GB, cond_stage_model.transformer.text_model.final_layer_norm.bias: 100%|██████████| 1131/1131 [00:06<00:00, 162.58it/s]                      


loaded weights in 6975.22 ms, 4.26 GB loaded at 0.61 GB/s


ram used:  5.71 GB, mid_block.resnets.1.conv2.bias                    : 100%|██████████| 340/340 [00:04<00:00, 81.01it/s]                    


loaded weights in 4200.28 ms, 5.71 GB loaded at 1.36 GB/s


In [9]:
state = get_state_dict(model)


In [11]:
state['alphas_cumprod'].numpy()

array([0.99915   , 0.998296  , 0.9974381 , 0.9965762 , 0.99571025,
       0.9948404 , 0.9939665 , 0.9930887 , 0.9922069 , 0.9913211 ,
       0.9904313 , 0.98953754, 0.9886398 , 0.9877381 , 0.9868324 ,
       0.98592263, 0.98500896, 0.9840913 , 0.9831696 , 0.982244  ,
       0.98131436, 0.9803808 , 0.97944313, 0.97850156, 0.977556  ,
       0.9766064 , 0.97565293, 0.9746954 , 0.9737339 , 0.9727684 ,
       0.97179896, 0.97082555, 0.96984816, 0.96886677, 0.9678814 ,
       0.96689206, 0.96589875, 0.9649015 , 0.96390027, 0.9628951 ,
       0.9618859 , 0.96087277, 0.95985574, 0.95883465, 0.9578097 ,
       0.95678073, 0.95574784, 0.954711  , 0.95367026, 0.9526256 ,
       0.9515769 , 0.95052433, 0.94946784, 0.94840735, 0.947343  ,
       0.94627476, 0.9452025 , 0.9441264 , 0.9430464 , 0.9419625 ,
       0.9408747 , 0.939783  , 0.9386874 , 0.93758786, 0.9364845 ,
       0.93537724, 0.9342661 , 0.9331511 , 0.9320323 , 0.9309096 ,
       0.929783  , 0.9286526 , 0.9275183 , 0.9263802 , 0.92523